<a href="https://colab.research.google.com/github/hit93/Advanced-Rag/blob/main/ANGEL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install smartapi-python pyotp pytz pandas logzero schedule

In [ ]:
from SmartApi import SmartConnect
import pyotp
import time
from datetime import datetime, timedelta
import pandas as pd
import schedule

# Your Angel One API credentials and TOTP
apikey = 'g3UdaiiP'
username = 'H178645'
pwd = '4523'
token = '7RRJ7OOVZ2APMRGUAJDODBKMIU'
exchange = "NSE"
timeperiod = "TEN_MINUTE"

# Nifty 50 tokens and symbols dictionary (partial; extend as needed)
nifty50_tokens = {
    1232: "GRASIM-EQ",
    1363: "HINDALCO-EQ",
    # 16675: "BAJAJFINSV-EQ",
    # 10604: "BHARTIARTL-EQ",
    # 21808: "SBILIFE-EQ",
    # 10940: "DIVISLAB-EQ",
    # 11287: "UPL-EQ",
    # 11723: "JSWSTEEL-EQ",
    # 1394: "HINDUNILVR-EQ",
    # 17963: "NESTLEIND-EQ",
    # 2475: "ONGC-EQ",
    # 3563: "ADANIGREEN-EQ",
    # 3787: "WIPRO-EQ",
    # 7229: "HCLTECH-EQ",
    # 3045: "SBIN-EQ",
    # 3456: "TATAMOTORS-EQ",
    # 547: "BRITANNIA-EQ",
    # 694: "CIPLA-EQ",
    # 2885: "RELIANCE-EQ",
    # 3103: "SHREECEM-EQ",
    # 467: "HDFCLIFE-EQ",
    # 5258: "INDUSINDBK-EQ",
    # 881: "DRREDDY-EQ",
    # 910: "EICHERMOT-EQ",
    # 10999: "MARUTI-EQ",
    # 11536: "TCS-EQ",
    # 13538: "TECHM-EQ",
    # 11532: "ULTRACEMCO-EQ",
    # 11630: "NTPC-EQ",
    # 15083: "ADANIPORTS-EQ",
    # 1922: "KOTAKBANK-EQ",
    # 20374: "COALINDIA-EQ",
    # 1333: "HDFCBANK-EQ",
    # 1348: "HEROMOTOCO-EQ",
    # 14977: "POWERGRID-EQ",
    # 1594: "INFY-EQ",
    # 2031: "M&M-EQ",
    # 11483: "LT-EQ",
    # 1624: "IOC-EQ",
    # 1660: "ITC-EQ",
    # 3432: "TATACONSUM-EQ",
    # 3499: "TATASTEEL-EQ",
    # 4963: "ICICIBANK-EQ",
    # 5900: "AXISBANK-EQ",
    # 236: "ASIANPAINT-EQ",
    # 317: "BAJFINANCE-EQ",
    # 3506: "TITAN-EQ",
    # 526: "BPCL-EQ"
}

# Create SmartConnect object
obj = SmartConnect(api_key=apikey)

def login():
    """
    Login to Angel One SmartAPI and get tokens.
    """
    data = obj.generateSession(username, pwd, pyotp.TOTP(token).now())
    auth_token = data['data']['jwtToken']
    refresh_token = data['data']['refreshToken']
    feed_token = obj.getfeedToken()
    print("Logged in successfully.")
    return auth_token, feed_token

def historical_data(exchange, token, from_date, to_date, interval):
    """
    Fetch historical candle data and return pandas DataFrame.
    """
    try:
        historicParam = {
            "exchange": exchange,
            "symboltoken": str(token),
            "interval": interval,
            "fromdate": from_date,
            "todate": to_date
        }
        api_response = obj.getCandleData(historicParam)
        data = api_response['data']
        columns = ['DateTime', 'Open', 'High', 'Low', 'Close', 'Volume']
        df = pd.DataFrame(data, columns=columns)
        df['DateTime'] = pd.to_datetime(df['DateTime'])
        df.set_index('DateTime', inplace=True)
        print(f"Fetched data for token {token} ({nifty50_tokens[token]}) with {len(df)} records.")
        return df
    except Exception as e:
        print(f"Error fetching data for token {token}: {e}")
        return pd.DataFrame()  # Return empty DataFrame on failure

def fetch_data():
    """
    Login and fetch historical data for all Nifty 50 tokens.
    Store each stock's DataFrame in a dict.
    """
    auth_token, feed_token = login()
    thirty_days_ago = datetime.now() - timedelta(days=30)
    from_date = thirty_days_ago.strftime("%Y-%m-%d %H:%M")
    to_date = datetime.now().strftime("%Y-%m-%d %H:%M")

    stock_data = {}
    for token, symbol in nifty50_tokens.items():
      df = historical_data(exchange, token, from_date, to_date, timeperiod)
      if df is not None and not df.empty:
          stock_data[symbol] = df  # Use symbol as key


    print(f"Fetched historical data for {len(stock_data)} Nifty 50 stocks.")
    return stock_data

def job():
    print(f"Job started at {datetime.now()}")
    data = fetch_data()

    # You can extend here by adding indicator calculations and signal generation
    print(f"Job completed at {datetime.now()}")
    print(data['GRASIM-EQ'])
if __name__ == "__main__":
    print("Starting scheduler to fetch data every 1 minute...")
    schedule.every(1).minutes.do(job)

    while True:
        schedule.run_pending()
        time.sleep(1)


[I 250927 10:55:30 smartConnect:124] in pool


Starting scheduler to fetch data every 1 minute...
Job started at 2025-09-27 10:56:30.471319
Logged in successfully.
Fetched data for token 1232 (GRASIM-EQ) with 828 records.
Fetched data for token 1363 (HINDALCO-EQ) with 828 records.
Fetched historical data for 2 Nifty 50 stocks.
Job completed at 2025-09-27 10:56:33.532759
                             Open    High     Low   Close  Volume
DateTime                                                         
2025-08-28 11:05:00+05:30  2784.6  2792.0  2783.8  2791.3   45381
2025-08-28 11:15:00+05:30  2791.3  2794.3  2790.0  2793.2    4592
2025-08-28 11:25:00+05:30  2793.5  2801.8  2791.5  2801.5    8168
2025-08-28 11:35:00+05:30  2801.5  2805.9  2800.0  2805.9    8155
2025-08-28 11:45:00+05:30  2805.7  2806.0  2802.5  2804.0   10885
...                           ...     ...     ...     ...     ...
2025-09-26 15:15:00+05:30  2746.6  2751.4  2745.0  2745.0   46593
2025-09-26 15:25:00+05:30  2745.2  2746.9  2739.1  2745.0   41523
2025-09-27 10:

KeyboardInterrupt: 